In [3]:
import importlib
from pathlib import Path
from pprint import pprint

import instructor
import openai
import pandas as pd

from cuery import utils
from cuery.work import tasks


GDRIVE = Path("~/Google Drive/Shared drives/Solutions").expanduser()

# Tasks

In [6]:
DATA_DIR = GDRIVE / "Research/future_of_work/inputs/ine_dirce_aggregated_by_activity.csv"
data = pd.read_csv(DATA_DIR).rename(
    columns={"Division": "division", "Actividad principal": "actividad"}
)
display(data)

n_examples = 5
context_df = data[["division", "actividad"]].iloc[:5]

context_dict = {
    "division": context_df["division"].tolist(),
    "actividad": context_df["actividad"].to_list(),
}

context_records = context_df.to_dict(orient="records")


,division,actividad,Estimated_Employees_2024,Median_YoY_Growth_pct
0,10 Industria de la alimentación,101 Procesado y conservación de carne y elabor...,134055,-2.4
1,"29 Fabricación de vehículos de motor, remolque...","293 Fabricación de componentes, piezas y acces...",97484,-0.2
2,43 Actividades de construcción especializada,"432 Instalaciones eléctricas, de fontanería y ...",303430,0.5
3,45 Venta y reparación de vehículos de motor y ...,451 Venta de vehículos de motor,102172,-0.5
4,46 Comercio al por mayor e intermediarios del ...,463 Comercio al por mayor de productos aliment...,325970,-0.9
5,46 Comercio al por mayor e intermediarios del ...,464 Comercio al por mayor de artículos de uso ...,228324,-2.0
6,"47 Comercio al por menor, excepto de vehículos...",471 Comercio al por menor en establecimientos ...,233708,1.3
7,"47 Comercio al por menor, excepto de vehículos...",477 Comercio al por menor de otros artículos e...,242341,-1.3
8,49 Transporte terrestre y por tubería,493 Otro transporte terrestre de pasajeros,142413,0.5
9,49 Transporte terrestre y por tubería,494 Transporte de mercancías por carretera y s...,273002,-1.5


In [10]:
pprint(tasks.DirceJobs)

In [7]:
dirce = await tasks.DirceJobs(context_df, model="gpt-3.5-turbo")
dirce

[Jobs(jobs=[Job(name='Técnico en control de calidad de alimentos', description='Responsable de garantizar la calidad de los alimentos procesados y conservados, así como de los productos cárnicos elaborados.', automation_potential=7, reason='Rutinario y basado en normativas establecidas.'), Job(name='Operario de producción de alimentos', description='Encargado de las tareas manuales en la línea de producción de alimentos, incluyendo el procesamiento de carne y productos cárnicos.', automation_potential=6, reason='Repetitivo y susceptible a la automatización de tareas físicas.'), Job(name='Técnico en procesamiento de alimentos', description='Responsable de supervisar y optimizar los procesos de procesamiento de carne y productos cárnicos para mejorar la eficiencia.', automation_potential=5, reason='Puede ser sustituido por sistemas automatizados en parte.')]),
 Jobs(jobs=[Job(name='Ingeniero de Automatización Industrial', description='Encargado de diseñar e implementar sistemas automátic

In [ ]:
def explode_list_models(models, field, to_pandas=True, **ids_vars):
    records = []
    for i, m in enumerate(models):
        for item in getattr(m, field):
            ids = {k: v[i] for k, v in ids_vars.items()}
            rec = ids | dict(item)
            records.append(rec)

    if to_pandas:
        return pd.DataFrame.from_records(records)

    return records


result = explode_list_models(
    dirce, "jobs", division=context_df["division"], actividad=context_df["actividad"]
)
result

,division,actividad,name,description,automation_potential,reason
0,10 Industria de la alimentación,101 Procesado y conservación de carne y elabor...,Técnico en control de calidad de alimentos,Responsable de garantizar la calidad de los al...,7,Rutinario y basado en normativas establecidas.
1,10 Industria de la alimentación,101 Procesado y conservación de carne y elabor...,Operario de producción de alimentos,Encargado de las tareas manuales en la línea d...,6,Repetitivo y susceptible a la automatización d...
2,10 Industria de la alimentación,101 Procesado y conservación de carne y elabor...,Técnico en procesamiento de alimentos,Responsable de supervisar y optimizar los proc...,5,Puede ser sustituido por sistemas automatizado...
3,"29 Fabricación de vehículos de motor, remolque...","293 Fabricación de componentes, piezas y acces...",Ingeniero de Automatización Industrial,Encargado de diseñar e implementar sistemas au...,9,Altamente repetitivo y basado en reglas predef...
4,"29 Fabricación de vehículos de motor, remolque...","293 Fabricación de componentes, piezas y acces...",Técnico de Mantenimiento de Robots Industriales,Responsable del mantenimiento y reparación de ...,8,Requiere habilidades técnicas y lógica secuenc...
5,"29 Fabricación de vehículos de motor, remolque...","293 Fabricación de componentes, piezas y acces...",Analista de Sistemas de Control de Calidad,Encargado de supervisar y mejorar los sistemas...,7,Automatizable mediante machine learning para d...
6,43 Actividades de construcción especializada,"432 Instalaciones eléctricas, de fontanería y ...",Técnico en Instalaciones Eléctricas,Encargado de la instalación y mantenimiento de...,7,High repetitive tasks and pattern recognition.
7,43 Actividades de construcción especializada,"432 Instalaciones eléctricas, de fontanería y ...",Fontanero de Obras,Responsable de la instalación y reparación de ...,8,Routine and manual precision tasks.
8,43 Actividades de construcción especializada,"432 Instalaciones eléctricas, de fontanería y ...",Inspector de Instalaciones,Encargado de verificar la correcta instalación...,6,Task based verification with rule adherence.
9,45 Venta y reparación de vehículos de motor y ...,451 Venta de vehículos de motor,Asesor de ventas de vehículos,Encargado de asesorar a los clientes en la com...,7,Tareas repetitivas y predecibles.
